# Here i will Extract the data from the CSV files and load it in Staging Layer called "Retail-Raw" Schema

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import time

In [66]:
!pip install psycopg2


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
!pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.1 MB 393.8 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.1 MB 357.2 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.1 MB 374.1 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/2.1 MB 409.6 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/2.1 MB 379.3 kB/s eta 0:00:06
   -- ------------------------------------- 0.2/2.1 MB 398.2 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/2.1 MB 420.8 kB/s eta 0:00:05
   ---- ----------------------------------- 0.2/2.1 MB 430.1 kB/s eta 0:00:05
   ---- ----------------------------------- 0.3/2.1 MB 436.5 kB/s eta 0:00:05
   ----- ---------------------------------- 0.3/2.1 MB 436.5 kB/s eta 0:00:05
   -


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def db_connection(user, password, host, port,db):
    engine= create_engine(
         f'postgresql://{user}:{password}@{host}:{port}/{db}'
    )
    engine.connect()
    return engine

In [3]:
engine= db_connection(user='postgres', password='postgres', host='localhost', port=5432 ,db='retail_raw')

## Here i will read the CSV as Chuncks, Chunck size is 100,000 record.
### And wirte it to as chuncks in Silver Layer 

In [4]:
df_iter = pd.read_csv(r'C:\Users\Mohammed\Desktop\retail-DWH\data\online_retail.csv', iterator=True, chunksize=100000)
df= next(df_iter)
df.head(n=0).to_sql(name='raw_data', con=engine, if_exists='replace', index=True, index_label='Id')
try:
    while True:
        s_time= pd.Timestamp.now()
        df.to_sql(name='raw_data', con=engine, if_exists='append', index=True, index_label='Id')
        e_time= pd.Timestamp.now()
        print("- Inserted:",df['InvoiceNo'].count(), f"Records, in {e_time - s_time}s")    
        df= next(df_iter)
except:
    print("Got Error")


- Inserted: 100000 Records, in 0 days 00:00:09.524145s
- Inserted: 100000 Records, in 0 days 00:00:09.099126s
- Inserted: 100000 Records, in 0 days 00:00:09.392231s
- Inserted: 100000 Records, in 0 days 00:00:10.014726s
- Inserted: 100000 Records, in 0 days 00:00:09.423145s
- Inserted: 41909 Records, in 0 days 00:00:03.249045s
Got Error


In [7]:
df= pd.read_csv(r'C:\Users\Mohammed\Desktop\retail-DWH\data\online_retail.csv')
df_dtypes = df.dtypes.reset_index()
df_dtypes.columns = ['Column_name', 'Data_Type']

# Print the DataFrame
print(df_dtypes)

  Column_name  Data_Type
0    InvoiceNo    object
1    StockCode    object
2  Description    object
3     Quantity     int64
4  InvoiceDate    object
5    UnitPrice   float64
6   CustomerID   float64
7      Country    object


In [8]:
import pandas as pd

# Dictionary containing column names and data types
data = {
    'Column_name': ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'],
    'Data_Type': ['object', 'object', 'object', 'int64', 'object', 'float64', 'float64', 'object']
}

# Create DataFrame from the dictionary
df_dtypes = pd.DataFrame(data)

print(df_dtypes)


   Column_name Data_Type
0    InvoiceNo    object
1    StockCode    object
2  Description    object
3     Quantity     int64
4  InvoiceDate    object
5    UnitPrice   float64
6   CustomerID   float64
7      Country    object


In [ ]:
# Merge the two DataFrames on the "Column_name" column
merged_df = pd.merge(result, df_dtypes, on='Column_name', suffixes=('_result', '_df_dtypes'))

# Compare the "Data_Type" columns
merged_df['Match'] = merged_df['Data_Type_result'] == merged_df['Data_Type_df_dtypes']

# Print the comparison result
print(merged_df)

# Check for columns that do not match
mismatched_columns = merged_df[merged_df['Match'] == False]['Column_name']
print("Mismatched columns:", mismatched_columns)

In [17]:
col_no= df.shape
col_no[1]
col_no= int(col_no[1])

if col_no != 8:
    print("got error in schema")
else:
    print("No Changes in Schema")

No Changes in Schema


: 